In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'ROOT'

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("-([0-9][0-9][0-9]|[0-9][0-9][0-9][0-9])e", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                pulsed_array.append(name)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def fit_data(x_data, y_data, time):
    max_points = []
    time_points = []
    gradients = []
    integral_areas = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        mp = []
        tp = []
        gr = []
        ia = []

        if x_pos == [] and y_pos == []:
            max_points.append(0)
            time_points.append(0)
            gradients.append(0)
            integral_areas.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):
                ifit1 = np.polyfit(x, y, 6)
                ifunc = np.poly1d(ifit1)
                ix_new = np.linspace(x[0], x[-1], 200)
                iy_new = ifunc(ix_new)
                
                min_point = min(iy_new)
                min_index = np.where(iy_new == min_point)[0][0]
                min_point_x = ix_new[min_index]
                
                for i in range(len(x)):
                    if list(x)[i] >= min_point_x and list(x)[i-1] < min_point_x:
                        x_index = i
                        break
                    else:
                        continue
                
                fit = np.polyfit(list(x)[:x_index], list(y)[:x_index], 2)
                func = np.poly1d(fit)
                x_new = np.linspace(x[0], x[x_index], 200)
                y_new = func(x_new)
            
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x, y)
                ax.plot(ix_new, iy_new)
                ax.plot(x_new, y_new)
                ax.set_title('Test')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                
                max_point = ifunc(x[min_index])
                mp.append(max_point)
                
                t_point = func(time)
                tp.append(t_point)
                
                grad = min(np.gradient(y_new, x_new))
                gr.append(grad)
                
                for i in range(len(x_new)):
                    if x_new[i] >= time and x_new[i-1] < time:
                        time_index = i
                        break
                    else:
                        continue
                
                area = sc.integrate.simpson(y_new[:time_index])
                ia.append(area)
            
            avg_mp = sum(mp)/len(mp)
            max_points.append(avg_mp)
            avg_tp = sum(tp)/len(tp)
            time_points.append(avg_tp)
            avg_gr = sum(gr)/len(gr)
            gradients.append(avg_gr)
            avg_ia = sum(ia)/len(ia)
            integral_areas.append(avg_ia)
        
    return max_points, time_points, gradients, integral_areas

def plotting_data(max_points, time_points, gradients, integral_areas, position, time):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(max_points))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(max_points[4:11]))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(time_points))
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(time_points)[4:11])
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(gradients))
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(gradients)[4:11])
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(integral_areas))
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(integral_areas)[4:11])
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
files = ["ndata_70um.root", "ndata_60um.root", "ndata_50um.root", "ndata_40um.root", "ndata_30um.root", "ndata_20um.root", "ndata_10um.root", "ndata_00um.root", "pdata_10um.root", "pdata_20um.root", "pdata_30um.root", "pdata_40um.root", "pdata_50um.root", "pdata_60um.root", "pdata_70um.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

max_points, time_points, gradients, integral_areas = fit_data(x_values, y_values, 3) 
plotting_data(max_points, time_points, gradients, integral_areas, [-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70], 3)

In [ ]:
def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("-([0-9][0-9][0-9]|[0-9][0-9][0-9][0-9])e", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                pulsed_array.append(name)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def fit_data(x_data, y_data, time):
    max_points = []
    time_points = []
    gradients = []
    integral_areas = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        mp = []
        tp = []
        gr = []
        ia = []

        if x_pos == [] and y_pos == []:
            max_points.append(0)
            time_points.append(0)
            gradients.append(0)
            integral_areas.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):
                min_point = min(list(y))
                x_index = list(y).index(min_point)
                
                fit = np.polyfit(list(x)[:x_index], list(y)[:x_index], 2)
                func = np.poly1d(fit)
                x_new = np.linspace(x[0], x[x_index], 200)
                y_new = func(x_new)
            
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x, y)
                ax.plot(x_new, y_new)
                ax.set_title('Test')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                
                mp.append(min_point)
                
                t_point = func(time)
                tp.append(t_point)
                
                grad = min(np.gradient(y_new, x_new))
                gr.append(grad)
                
                for i in range(len(x_new)):
                    if x_new[i] >= time and x_new[i-1] < time:
                        time_index = i
                        break
                    else:
                        continue
                
                area = sc.integrate.simpson(y_new[:time_index])
                ia.append(area)
            
            avg_mp = sum(mp)/len(mp)
            max_points.append(avg_mp)
            avg_tp = sum(tp)/len(tp)
            time_points.append(avg_tp)
            avg_gr = sum(gr)/len(gr)
            gradients.append(avg_gr)
            avg_ia = sum(ia)/len(ia)
            integral_areas.append(avg_ia)
        
    return max_points, time_points, gradients, integral_areas

def plotting_data(max_points, time_points, gradients, integral_areas, position, time):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(max_points))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(max_points[4:11]))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(time_points))
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(time_points)[4:11])
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(gradients))
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(gradients)[4:11])
    ax.set_title('Minimum Gradient against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(integral_areas))
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[4:11], np.abs(integral_areas)[4:11])
    ax.set_title(f'Integral area against position at time = {time}ns')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
files = ["ndata_70um.root", "ndata_60um.root", "ndata_50um.root", "ndata_40um.root", "ndata_30um.root", "ndata_20um.root", "ndata_10um.root", "ndata_00um.root", "pdata_10um.root", "pdata_20um.root", "pdata_30um.root", "pdata_40um.root", "pdata_50um.root", "pdata_60um.root", "pdata_70um.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

max_points, time_points, gradients, integral_areas = fit_data(x_values, y_values, 3) 
plotting_data(max_points, time_points, gradients, integral_areas, [-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70], 3)

In [ ]:
def pulsedisplay(file, pulse_graph):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    event = detector.Get(pulse_graph)
    
    data_x = event.GetX()
    data_y = event.GetY()

    integrate = sc.integrate.cumulative_trapezoid(data_y, data_x)
    return event, data_x, data_y, integrate

def collect_counts(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("-1e", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                #print(name)
                pulsed_array.append(name)
    return pulsed_array

In [ ]:
e_40 = collect_counts("e_40um.root")
e_20 = collect_counts("e_20um.root")
e_zero = collect_counts("e_0um.root")
e_n20 = collect_counts("e_20num.root")
e_n40 = collect_counts("e_40num.root")
a = [e_n40, e_n20, e_zero, e_20, e_40]
print(a)

filess = ["e_40num.root", "e_20num.root", "e_0um.root", "e_20um.root", "e_40um.root"]
avg_mine = []
for d, f in zip(a, filess):
    minimum = []
    for i in d:
        e, x, y, integrated = pulsedisplay(f, i)
        minimum.append(min(y))
    avg = sum(minimum)/len(minimum)
    avg_mine.append(np.abs(avg))
print(avg_mine)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot([-40,-20,0,20,40], avg_mine)

In [ ]:
z = np.linspace(0, 0.01, 100)
y = -600000*z + 4500
print(y)
print(sum(y)/len(y))

In [ ]:
c1 = TCanvas( 'c1', 'Testing', 200, 10, 700, 900 )
c1.SetFillColor( 18 )
c1.SetGrid()

gr = TGraph( a[2] )
gr.SetLineColor( 2 )
gr.SetLineWidth( 4 )
gr.SetMarkerColor( 4 )
gr.SetMarkerStyle( 21 )
gr.SetTitle( a[2].GetTitle() )
gr.GetXaxis().SetTitle( f'{a[2].GetXaxis().GetTitle()}' )
gr.GetYaxis().SetTitle( f'{a[2].GetYaxis().GetTitle()}' )
gr.Draw( 'ACP' )

c1.Update()
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderSize( 12 )
c1.Modified()
c1.Update()

e1, x1, y1, integrated1 = pulsedisplay("modules2.root", 'pulse_ev1_px268-28')

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(list(x1)[580:650], list(y1)[580:650])

In [ ]:
e, x, y, integrated = pulsedisplay("modules.root", "pulse_ev4_px257-1")

c1 = TCanvas( 'c1', 'Testing', 200, 10, 700, 900 )
c1.SetFillColor( 18 )
c1.SetGrid()

gr = TGraph( e )
gr.SetLineColor( 2 )
gr.SetLineWidth( 4 )
gr.SetMarkerColor( 4 )
gr.SetMarkerStyle( 21 )
gr.SetTitle( e.GetTitle() )
gr.GetXaxis().SetTitle( f'{e.GetXaxis().GetTitle()}' )
gr.GetYaxis().SetTitle( f'{e.GetYaxis().GetTitle()}' )
gr.Draw( 'ACP' )

c1.Update()
c1.GetFrame().SetFillColor( 21 )
c1.GetFrame().SetBorderSize( 12 )
c1.Modified()
c1.Update()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(x, y)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(list(x)[:-1], integrated)
ax.set_title('Integrated Pulse')
ax.set_ylabel('Cumulative Charge (e)')
ax.set_xlabel('Time (ns)')

In [ ]:
e, x, y, integrated = pulsedisplay("modules.root", "current_ev4_px257-1")

c2 = TCanvas( 'c2', 'Testing', 200, 10, 700, 900 )
c2.SetFillColor( 18 )
c2.SetGrid()

gr1 = TGraph( e )
gr1.SetLineColor( 2 )
gr1.SetLineWidth( 4 )
gr1.SetMarkerColor( 4 )
gr1.SetMarkerStyle( 21 )
gr1.SetTitle( e.GetTitle() )
gr1.GetXaxis().SetTitle( f'{e.GetXaxis().GetTitle()}' )
gr1.GetYaxis().SetTitle( f'{e.GetYaxis().GetTitle()}' )
gr1.Draw( 'ACP' )

c2.Update()
c2.GetFrame().SetFillColor( 21 )
c2.GetFrame().SetBorderSize( 12 )
c2.Modified()
c2.Update()

new_x, new_y = [], []
for v, u in zip(x, y):
    new_x.append(v)
    new_y.append(u)

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
#ax.plot(x, np.gradient(y, x))
#print(max(np.gradient(y, x)))
#ax.set_title('Differentiated Pulse')
#ax.set_ylabel('Current (e)')
#ax.set_xlabel('Time (ns)')

y_max = min(y)
i = list(y).index(y_max)
x_max = x[i]
#print(x_max)

fit1 = np.polyfit(new_x[65:i+2], new_y[65:i+2], 3)
func = np.poly1d(fit1)
x_new = np.linspace(new_x[65], new_x[i+2], 200)
y_new = func(x_new)
#print(func)

print(func(1))

#fit = sc.optimize.curve_fit(fit1, new_x[65:i+4], new_y[65:i+4])

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(abs(np.array(x)[:i+65]), -abs(np.array(y)[:i+65]))
ax.plot(x_new, y_new)
ax.set_title('New')
ax.set_ylabel('Current (e)')
ax.set_xlabel('Time (ns)')
print(min(np.gradient(list(y)[:i], list(x)[:i])))

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(x_new, np.gradient(y_new, x_new))
ax.set_title('Differentiated Pulse')
ax.set_ylabel('Current (e)')
ax.set_xlabel('Time (ns)')

print(min(np.gradient(y_new, x_new)))

y_int = []
for val in list(y):
    if val>0:
        continue
    else:
        y_int.append(val)

print(sc.integrate.simpson(y_new))
#print(np.trapz(np.abs(list(y))[:i]))

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    errors = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                for i in range(len(x)):
                    if x[i] >= 0.6 and x[i-1] < 0.6:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                hole_cont = np.abs(y[hole_index])
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, np.abs(y), label='Real Data')
                #ax.plot(x_pdf, y_pdf)
                ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = max(np.abs(y))
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                err = (max_point_raw - np.abs(y[time_index_r]))/2
                errors.append(err)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return max_points_r, time_points_r, max_points_c, time_points_c, errors

def getting_e_field(time_points_r, time_points_c, no_charge, errors, position, time):
    e_field_tot_r = []
    e_field_tot_c = []
    e_field_tot_err = []
    
    v_m = 1.53*(10**9)*(258.15**(-0.87))
    E_c = 1.01*(258.15**1.55)
    B = 0.0257*(258.15**0.66)
    
    def mu_e_field(x):
        return np.abs(((-50222+49533)/(0.0185+0.0389))*x - 50000)
        
    for n, t, p in zip(no_charge, time_points_c, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_c = val/(charge*weighting_field*mu_si)
        e_field_tot_c.append(e_field_c)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        
    for n, t, p in zip(no_charge, errors, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_err = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_err)

    def func1(x, A, B):
        return A*x + B
    
    popt, pcov = sc.optimize.curve_fit(func1, position[2:10], e_field_tot_c[2:10])
    fit1 = func1(np.array(position[2:10]), popt[0], popt[1])
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position[2:10], e_field_tot_r[2:10])
    fit2 = func1(np.array(position[2:10]), poptr[0], poptr[1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[2:10], mu_e_field(np.array(position[2:10])/1000), 'black', label='Actual Electric Field')
   
    #ax.plot(position[2:10], fit1, 'r--', label='Straight Line Fit for Convoluted Data')
    ax.errorbar(position[2:10], fit1, yerr=errors[2:10], label='Straight Line Fit for Convolution Data', color='purple')
    ax.plot(position[2:10], e_field_tot_c[2:10], 'green', label=' Convolution Electric Field Data')
    
    #ax.plot(position[2:10], fit2, 'g--', label='Straight Line Fit for Real Data')
    ax.errorbar(position[2:10], fit2, yerr=e_field_tot_err[2:10], label='Straight Line Fit for Electric Field Data', color='blue')
    ax.plot(position[2:10], e_field_tot_r[2:10], 'red', label='Electric Field Data')
    
    ax.set_title(f'Electric Field against Position at t={time}ns')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (um)')
    ax.legend()

def plotting_data(max_points_r, time_points_r, max_points_c, time_points_c, position, time):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, max_points_r, label= 'real')
    ax.plot(position, max_points_c, label= 'convolution')
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    ax.legend()
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, time_points_r, label= 'real')
    ax.plot(position, time_points_c, label= 'convolution')
    ax.set_title(f'Time Points against position at t={time}')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    ax.legend()
    
files = ["data_10ps_n50.root", "data_10ps_n40.root", "extra_data_n35.root", "data_10ps_n30.root", "extra_data_n25.root","data_10ps_n20.root", "extra_data_n15.root", "data_10ps_n10.root", "extra_data_n5.root", "data_10ps_0.root", "data_10ps_p10.root", "data_10ps_p20.root", "data_10ps_p30.root", "data_10ps_p40.root", "data_10ps_p50.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

max_points_r, time_points_r, max_points_c, time_points_c, errors = fit_convolve_data(x_values, y_values, 0.06) 

getting_e_field(time_points_r, time_points_c, pulse_names, errors, [50, 40, 35, 30, 25, 20, 15, 10, 5, 0, -10, -20, -30, -40, -50], 0.06)
    
#plotting_data(max_points_r, time_points_r, max_points_c, time_points_c, [50, 40, 35, 30, 25, 20, 15, 10, 5, 0, -10, -20, -30, -40, -50], 0.1)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def fit_overbias_slow(x_data, y_data, time):
    max_points = []
    time_points = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        if x_pos == [] and y_pos == []:
            max_points.append(0)
            time_points.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  
                
                def gauss(x_c, H, A, x0, sigma):
                    x = np.array(x_c)
                    return H + A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

                def gauss_fit(x_c, y_c):
                    x = np.array(x_c)
                    y = np.array(y_c)
                    mean = sum(x * y) / sum(y)
                    sigma = np.sqrt(sum(y * (x - mean) ** 2) / sum(y))
                    popt, pcov = sc.optimize.curve_fit(gauss, x, y, p0=[min(y), max(y), mean, sigma])
                    return popt, pcov
                
                y_fit = gauss(x, *gauss_fit(x, y)[0])
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                plt.plot(x, y_fit, label='fit')
                ax.plot(x, y,'--r')
                ax.set_title('Test')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                
                max_point = np.abs(min(y_fit))
                max_points.append(max_point)
                
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index = i
                        break
                    else:
                        continue
                    
                time_point = np.abs(y_fit[time_index])
                time_points.append(time_point)
        
    return max_points, time_points

def getting_e_field(time_points_c, no_charge, position):
    e_field_tot = []
    for n, t in zip(no_charge, time_points_c):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = 186    #220 #152
        e_field = val/(charge*weighting_field*mu_si)
        e_field_tot.append(e_field)
    
    print(len(e_field_tot))
        
    fit = np.polyfit(position, e_field_tot, 2)
    func = np.poly1d(fit)
    x_new = np.linspace(position[0], position[-1], 200)
    y_new = func(x_new)
    
    print(func)
    
    def func1(x, A, B, C):
        return A*(x**2) + B*x + C
    
    popt, pcov = sc.optimize.curve_fit(func1, position, e_field_tot) #p0=[-1.407, 22.63, 15710])
    print(popt)
    y_data = func1(np.array(position), popt[0], popt[1], popt[2])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, e_field_tot, label='Data')
    ax.plot(position, y_data, 'r--', label='Data Fit')
    #ax.errorbar(position, y_data, yerr=np.sqrt(np.diag(pcov)[1]))
    #ax.plot(x_new, y_new)
    ax.set_title(f'Electric Field against Position Using Maximum Current')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (um)')
    ax.legend()

def plotting_data(max_points_r, position):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, max_points_r)
    ax.set_title('Maximum Current against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    ax.legend()
    
files = ["overbias_1ns_n40.root", "overbias_1ns_n30.root", "overbias_1ns_n20.root", "overbias_1ns_n10.root", "overbias_1ns_0.root", "overbias_1ns_p10.root", "overbias_1ns_p20.root", "overbias_1ns_p30.root", "overbias_1ns_p40.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

max_points, time_points = fit_overbias_slow(x_values, y_values, 3) 

getting_e_field(time_points, pulse_names, [40, 30, 20, 10, 0, -10, -20, -30, -40])
    
plotting_data(time_points, [40, 30, 20, 10, 0, -10, -20, -30, -40])

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    return pulsed_array, data_x, data_y


#Code to firstly find the peaks of the fits for the pulses

def fit_data_less_overbias(x_data, y_data, time, p):
    max_points = []
    time_points_r = []
    time_points_c = []
    velocity = []
    
    for x_pos, y_pos, position in zip(x_data, y_data, p):

        if x_pos == [] and y_pos == []:
            max_points.append(0)
            time_points.append(0)

            
        else:
            for x, y in zip(x_pos, y_pos):  
                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.07, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                for i in range(len(x)):
                    if x[i] >= 4.5 and x[i-1] < 4.5:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 3 and x_convolve[i-1] < 3:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                hole_cont = np.abs(y[hole_index])
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
            
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, np.abs(y), label= 'Real Data')
                ax.set_title(f'Current Pulse at {50+position}um from the Electrode')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point = min(y)
                max_point1 = max(y_convolve)
                max_points.append(max_point1)
                
                max_index = np.where(np.array(y) == max_point)[0][0]
                total_time = x[max_index]
                v = (50-position)*(10**(-4))/(total_time*(10**(-9)))
                velocity.append(v)
                
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                
                t_point_r = np.abs(y[time_index_r]) - hole_cont
                time_points_r.append(t_point_r)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                t_point_c = np.abs(y_convolve[time_index_c]) - hole_cont_c
                time_points_c.append(t_point_c)
        
    return max_points, time_points_r, time_points_c, velocity

def getting_e_field(time_points_c, time_points_r, velocity, no_charge, position, time):
    e_field_tot_ic = []
    e_field_tot_ir = []
    e_field_tot_v = []
    
    v_m = 1.53*(10**9)*(258.15**(-0.87))
    E_c = 1.01*(258.15**1.55)
    B = 0.0257*(258.15**0.66)
    
    def mu_e_field(x):
        return np.abs(((-1395+785.2)/(0.03305+0.0179))*x - 1000)
        
    for n, t, v, p in zip(no_charge, time_points_c, velocity, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_fieldc = val/(charge*weighting_field*mu_si)
        e_field_tot_ic.append(e_fieldc)
        
        e_field_v = v/mu_si
        e_field_tot_v.append(e_field_v)
        
    for n, t, v, p in zip(no_charge, time_points_r, velocity, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_fieldr = val/(charge*weighting_field*mu_si)
        e_field_tot_ir.append(e_fieldr)
        
    def func1(x, A, B):
        return A*x + B
    
    poptc, pcovc = sc.optimize.curve_fit(func1, position[:16], e_field_tot_ic[:16])
    fit1 = func1(np.array(position)[:16], poptc[0], poptc[1])
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position[:16], e_field_tot_ir[:16])
    fit3 = func1(np.array(position)[:16], poptr[0], poptr[1])
    
    poptv, pcovv = sc.optimize.curve_fit(func1, position, e_field_tot_v)
    fit2 = func1(np.array(position), poptv[0], poptv[1])
        
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position[:16], e_field_tot_ir[:16], 'red', label= f'Electric Field from Current Data')
    ax.errorbar(position[:16], fit3, yerr=np.sqrt(np.diag(pcovr)[1]), color= 'blue', label='Straight Line Fit for Current Data')
    
    ax.plot(position[:16], mu_e_field(np.array(position[:16])/1000), 'black', label='Actual Electric Field')
    
    ax.plot(position[:16], e_field_tot_ic[:16], 'green', label= f'Electric Field from Convolution')
    ax.errorbar(position[:16], fit1, yerr=np.sqrt(np.diag(pcovc)[1]), color= 'purple', label='Straight Line Fit for Convolution')
    
    e_field_tot_v.pop(1)
    position.pop(1)
    poptv, pcovv = sc.optimize.curve_fit(func1, position, e_field_tot_v)
    fit2 = func1(np.array(position), poptv[0], poptv[1])
    
    #ax.plot(position, e_field_tot_v, 'orange', label= 'Electric Field from Velocity Calculation')
    #ax.plot(position, fit2, 'r', label='Straight Line Fit for Velocity Based Plot')
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovv)[1]), ecolor='r')
    ax.set_title(f'Electric Field against Laser Position for edge-TCT')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (um)')
    ax.legend()
        

def plotting_data(max_points, time_points, velocity, position, time):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(max_points))
    ax.set_title('Maximum Points against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(time_points))
    ax.set_title(f'Points at time = {time}ns against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, np.abs(velocity))
    ax.set_title(f'Velocities against position')
    ax.set_ylabel('Current (uA)')
    ax.set_xlabel('Position (um)')
    
files = ["less_overbias_n45.root", "less_overbias_n40.root", "less_overbias_n35.root", "less_overbias_n30.root", "less_overbias_n25.root", "less_overbias_n20.root", "less_overbias_n15.root", "less_overboas_n10.root", "less_overbias_n5.root", "less_overbias_0.root", "less_overbias_p5.root", "less_overbias_p10.root", "less_overbias_p15.root", "less_overbias_p20.root", "less_overbias_p25.root", "less_overbias_p30.root", "less_overbias_p35.root", "less_overbias_p40.root", "less_overbias_p45.root"]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(list(x_values[1][0]), list(y_values[1][0]), label= '-50')
ax.plot(list(x_values[2][0]), list(y_values[2][0]), label = '-40')
ax.plot(list(x_values[3][0]), list(y_values[3][0]), label = '-30')
ax.plot(list(x_values[4][0]), list(y_values[4][0]), label = '-20')
ax.plot(list(x_values[5][0]), list(y_values[5][0]), label = '-10')
ax.plot(list(x_values[6][0]), list(y_values[6][0]), label = '0')
ax.plot(list(x_values[7][0]), list(y_values[7][0]), label = '10')
ax.plot(list(x_values[8][0]), list(y_values[8][0]), label = '20')
ax.plot(list(x_values[9][0]), list(y_values[9][0]), label = '30')
ax.plot(list(x_values[10][0]), list(y_values[10][0]), label = '40')
ax.plot(list(x_values[11][0]), list(y_values[11][0]), label = '50')
ax.legend()
ax.set_title(f'Integral area against position at time = ns')
ax.set_ylabel('Current (uA)')
ax.set_xlabel('Position (um)')
    
print(list(y_dat[0]))
#for i in range(len(x_dat[0])):
#    print(y_dat[0][i]-y_dat[0][i-1])

max_points, time_points_r, time_points_c, velocity = fit_data_less_overbias(x_values, y_values, 0.07, [45, 40, 35, 30, 25, 20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30, -35, -40, -45]) 

getting_e_field(time_points_c, time_points_r, velocity, pulse_names, [45, 40, 35, 30, 25, 20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30, -35, -40, -45], 0.07)
    
plotting_data(max_points, time_points_c, velocity, [45, 40, 35, 30, 25, 20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30, -35, -40, -45], 0.07)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = hole.GetBinContent(7)
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(y_list)
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, hy, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                hole_cont = ((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, np.abs(y), label='Real Data')
                #ax.plot(x_pdf, y_pdf)
                ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = max(np.abs(y))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return max_points_r, time_points_r, max_points_c, time_points_c

def getting_e_field(time_points_r, time_points_c, no_charge, error, position, time):
    e_field_tot_r = []
    e_field_tot_c = []
    e_field_tot_err = []
    
    v_m = 1.53*(10**9)*(258.15**(-0.87))
    E_c = 1.01*(258.15**1.55)
    B = 0.0257*(258.15**0.66)
    
    def mu_e_field(x):
        return np.abs(((50464-49660)/(0.0386+0.0284))*x + 49404)
        
    for n, t, p in zip(no_charge, time_points_c, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_c = val/(charge*weighting_field*mu_si)
        e_field_tot_c.append(e_field_c)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_e)

    def func1(x, A, B):
        return A*x + B
    
    popt, pcov = sc.optimize.curve_fit(func1, position, e_field_tot_c)
    fit1 = func1(np.array(position), popt[0], popt[1])
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position, e_field_tot_r)
    fit2 = func1(np.array(position), poptr[0], poptr[1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, mu_e_field(np.array(position)/1000), 'black', label='Actual Electric Field')
     
    ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    
    ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    #ax.plot(position[2:10], fit2, 'g--', label='Straight Line Fit for Real Data')
    #ax.errorbar(position, fit2, yerr=e_field_tot_err, label='Straight Line Fit for Electric Field Data', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    ax.set_title(f'Electric Field against Position at t={time}ns')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (um)')
    ax.legend()

 
files = ["no_peak_0.root", "no_peak_5.root", "no_peak_10.root", "no_peak_15.root", "no_peak_20.root","no_peak_25.root", "no_peak_30.root", "no_peak_35.root", "no_peak_40.root"]
x_values, y_values, pulse_names, hole_dat, error = [], [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)
    error.append((((diff*sc.constants.e)/(0.01*(10**(-9))))*(10**6)))

max_points_r, time_points_r, max_points_c, time_points_c = fit_convolve_data(x_values, y_values, 0.09, hole_dat) 
getting_e_field(time_points_r, time_points_c, pulse_names, error, [0, -5, -10, -15, -20, -25, -30, -35, -40], 0.09)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = hole.GetBinContent(7)
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(y_list)
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, hy, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                for i in range(len(x)):
                    if x[i] >= 0.55 and x[i-1] < 0.55:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                hole_cont = np.abs(y[hole_index])
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
                #fig = plt.figure()
                #ax = fig.add_subplot(1,1,1)
                #ax.plot(x_convolve, y_convolve, label='Convolution')
                #ax.plot(x, np.abs(y), label='Real Data')
                #ax.plot(x_pdf, y_pdf)
                #ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                #ax.set_ylabel('Current (uA)')
                #ax.set_xlabel('Time (ns)')
                #ax.legend()
                
                max_point_raw = max(np.abs(y))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return max_points_r, time_points_r, max_points_c, time_points_c

def getting_e_field(time_points_r, time_points_c, no_charge, error, position, time):
    e_field_tot_r = []
    e_field_tot_c = []
    e_field_tot_err = []
    
    v_m = 1.53*(10**9)*(258.15**(-0.87))
    E_c = 1.01*(258.15**1.55)
    B = 0.0257*(258.15**0.66)
    
    def mu_e_field(x):
        return np.abs(((50464-49660)/(0.0386+0.0284))*x + 49404)
        
    for n, t, p in zip(no_charge, time_points_c, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_c = val/(charge*weighting_field*mu_si)
        e_field_tot_c.append(e_field_c)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((mu_e_field(p/1000)/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_e)

    def func1(x, A, B):
        return A*x + B
    
    popt, pcov = sc.optimize.curve_fit(func1, position, e_field_tot_c)
    fit1 = func1(np.array(position), popt[0], popt[1])
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position, e_field_tot_r)
    fit2 = func1(np.array(position), poptr[0], poptr[1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, mu_e_field(np.array(position)/1000), 'black', label='Actual Electric Field')
     
    ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    
    ax.errorbar(position, fit2, yerr=e_field_tot_err, label='Straight Line Fit for Electric Field Data', color='blue')
    ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    ax.set_title(f'Electric Field against Position at t={time}ns')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (um)')
    ax.legend()

 
files = ["no_peak_0.root", "no_peak_5.root", "no_peak_10.root", "no_peak_15.root", "no_peak_20.root","no_peak_25.root", "no_peak_30.root", "no_peak_35.root", "no_peak_40.root"]
x_values, y_values, pulse_names, hole_dat, error = [], [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)
    error.append((((diff*sc.constants.e)/(0.01*(10**(-9))))*(10**6)))

max_points_r, time_points_r, max_points_c, time_points_c = fit_convolve_data(x_values, y_values, 0.08, hole_dat) 
getting_e_field(max_points_r, time_points_c, pulse_names, error, [0, -5, -10, -15, -20, -25, -30, -35, -40], 0.08)

In [ ]:
myFile = TFile.Open("modules.root", "read")
location = myFile.Get("DetectorHistogrammer")
location1 = location.Get("detector1")
hits = location1.Get("hit_map;1")
print(type(hits))
a = hits.GetBinContent
print(a)

h2 = TCanvas( 'h2', 'Testing', 200, 10, 700, 900 )
h2.SetFillColor( 18 )
h2.SetGrid()

gr = TH2D( hits )
gr.SetLineColor( 2 )
gr.SetLineWidth( 4 )
gr.SetMarkerColor( 4 )
gr.SetMarkerStyle( 21 )
gr.SetTitle( 'a simple graph' )
gr.GetXaxis().SetTitle( 'X title' )
gr.GetYaxis().SetTitle( 'Y title' )
gr.GetZaxis().SetTitle(' Z title' )
gr.Draw( 'ACP' )

h2.Update()
h2.GetFrame().SetFillColor( 21 )
h2.GetFrame().SetBorderSize( 12 )
h2.Modified()
h2.Update()